In [2]:
import pandas as pd
import datetime as dt
import copy
from pandas.plotting import register_matplotlib_converters
import copy
import os
import netCDF4 as nc4            # to work with netCDFs
import time

register_matplotlib_converters()
os.getcwd()

'/home/waniomar/Office/Berkeley/Scripts/sde/python'

In [10]:
watershed = ['1-EastRiver', '2-DryCreek','3-Sagehen','4-HJAndrews','5-Baltimore',
             '6-Bonanza','7-CCE','8-CentralArizona','9-Coweeta','10-Florida',
             '11-Georgia','12-Harvard','13-Hubbard','14-Jornada','15-Kellogg',
             '16-KonzaPrairie','17-NGA','18-PlumIsland','19-Sevilleta','20-Boulder',
             '21-Catalina','22-Jemez','23-WhiteClay','24-Luquillo','25-Reynolds',
             '26-ShaleHill','27-SanJoaquin','28-Providence','29-Wolverton','30-Calhoun']

In [11]:
path = '/home/waniomar/Office/Berkeley/Data_Zhang/chosen-master/DataProduct'

In [13]:
ws='EastRiver'
File=ws+'_NetCDF.nc'
ncdf = nc4.Dataset(path +'/'+ File, 'r')

In [6]:
# date - time info
def get_time_range(watershed_number):
    ws=watershed[watershed_number-1].split('-')[1]
    print(ws)
    File=ws+'_NetCDF.nc'
    ncdf = nc4.Dataset(path +'/'+ File, 'r')
    dt_lst=[pd.to_datetime(i) for i in ncdf.variables['Datetime'][:]]
    ncdf.close()
    print(ws+' data is from', dt_lst[0], 'to',dt_lst[-1])
    #return [str(dt_lst[0]),str(dt_lst[1])]
    # station info
    
def get_station_name(watershed_number):
    ws=watershed[watershed_number-1].split('-')[1]
    print(ws)
    File=ws+'_NetCDF.nc'
    ncdf = nc4.Dataset(path +'/'+ File, 'r')
    stn_names=ncdf.variables['Grid'][:]
    ncdf.close()
    return stn_names
def get_station_latitude(watershed_number):
    ws=watershed[watershed_number-1].split('-')[1]
    print(ws)
    File=ws+'_NetCDF.nc'
    ncdf = nc4.Dataset(path +'/'+ File, 'r')
    stn_names=ncdf.variables['Grid'].Latitude
    ncdf.close()
    return stn_names
def get_station_longitude(watershed_number):
    ws=watershed[watershed_number-1].split('-')[1]
    print(ws)
    File=ws+'_NetCDF.nc'
    ncdf = nc4.Dataset(path +'/'+ File, 'r')
    stn_names=ncdf.variables['Grid'].Longitude
    ncdf.close()
    return stn_names
def get_station_elevation(watershed_number): # in meter
    ws=watershed[watershed_number-1].split('-')[1]
    print(ws)
    File=ws+'_NetCDF.nc'
    ncdf = nc4.Dataset(path +'/'+ File, 'r')
    stn_names=ncdf.variables['Grid'].Elevation_m
    ncdf.close()
    return stn_names

# extract data table
def get_data(watershed_number,variable_names_lst=[]): # variable_names_lst=[] to get all the data
    ws=watershed[watershed_number-1].split('-')[1]
    print(ws)
    File=ws+'_NetCDF.nc'
    ncdf = nc4.Dataset(path +'/'+ File, 'r')
    keys_lst=list(ncdf.variables.keys())
    len_keys=len(keys_lst)
    print("Hydrometeorological variables in this watershed are:")
    all_variables=keys_lst[2:int(len_keys/2)]
    print(all_variables)
    if variable_names_lst==[]:
        for i,var in enumerate(all_variables):
            indexUnique = pd.date_range(str(pd.to_datetime(ncdf.variables['Datetime'][:][0])), 
                                        str(pd.to_datetime(ncdf.variables['Datetime'][:][-1])))  
            var_df=pd.DataFrame(ncdf.variables[var][:], index=indexUnique, 
                                columns = [ncdf.variables[var].names])
            if i==0:
                former_df=var_df
            else:
                former_df=pd.concat([former_df,var_df],axis=1,join='outer')
    else:
        for i,var in enumerate(variable_names_lst):
            indexUnique = pd.date_range(str(pd.to_datetime(ncdf.variables['Datetime'][:][0])), 
                                        str(pd.to_datetime(ncdf.variables['Datetime'][:][-1])))  
            
            if ncdf.variables[var][:].shape[1]==1:
                var_df=pd.DataFrame(ncdf.variables[var][:], index=indexUnique, columns = [ncdf.variables[var].names])
            else:
                var_df=pd.DataFrame(ncdf.variables[var][:], index=indexUnique, columns = ncdf.variables[var].names)
            
            if i==0:
                former_df=var_df
            else:
                former_df=pd.concat([former_df,var_df],axis=1,join='outer')      
    ncdf.close()
    former_df.index.rename('DateTime',inplace = True)
    return former_df